# Notebook de Teste de Modelos - Trackie

Este notebook é usado para testar os serviços de visão computacional (YOLO, MiDaS, DeepFace) de forma isolada.

**Instruções:**
1.  Certifique-se de ter todas as dependências instaladas (`pip install -r requirements.txt`).
2.  Coloque uma imagem de teste em `data/assets/test_image.jpg`.
3.  Execute as células abaixo para ver os resultados de cada modelo.

In [ ]:
import sys
import os
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Adiciona o diretório 'src' ao path para que possamos importar nossos módulos
# Assumindo que o notebook está na pasta 'notebooks' na raiz do projeto
project_root = Path(os.getcwd()).parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Importa nossos serviços e configurações
from trackie.config.settings import settings
from trackie.services.vision.object_detector import ObjectDetector
from trackie.services.vision.depth_estimator import DepthEstimator
from trackie.services.vision.face_recognizer import FaceRecognizer

# Converte as configurações para um dicionário para facilitar o uso
settings_dict = settings.dict()

# Caminho para a imagem de teste
TEST_IMAGE_PATH = project_root / "data/assets/test_image.jpg"

print(f"Raiz do projeto: {project_root}")
print(f"Imagem de teste: {TEST_IMAGE_PATH}")

# Helper para exibir imagens
def show_image(img, title="Image", cmap=None):
    # Converte de BGR (OpenCV) para RGB (Matplotlib)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

## 1. Teste do Detector de Objetos (YOLO)

In [ ]:
# Carrega o serviço de detecção de objetos
try:
    detector = ObjectDetector(settings_dict)
    print("ObjectDetector carregado com sucesso.")
except Exception as e:
    print(f"Falha ao carregar ObjectDetector: {e}")
    detector = None

if detector:
    # Carrega a imagem de teste
    test_image = cv2.imread(str(TEST_IMAGE_PATH))
    if test_image is None:
        print(f"Erro: Não foi possível carregar a imagem de {TEST_IMAGE_PATH}")
    else:
        # Realiza a detecção
        detections = detector.detect(test_image)
        
        # Desenha as detecções na imagem
        annotated_image = test_image.copy()
        if detections:
            annotated_image = detections[0].plot() # O método .plot() do ultralytics é útil para visualização
        
        print(f"Detecções encontradas: {len(detections[0].boxes) if detections else 0}")
        show_image(annotated_image, title="Resultados da Detecção YOLO")

## 2. Teste do Estimador de Profundidade (MiDaS)

In [ ]:
# Carrega o serviço de estimativa de profundidade
try:
    estimator = DepthEstimator(settings_dict)
    print("DepthEstimator carregado com sucesso.")
except Exception as e:
    print(f"Falha ao carregar DepthEstimator: {e}")
    estimator = None

if estimator:
    # Carrega a imagem de teste
    test_image = cv2.imread(str(TEST_IMAGE_PATH))
    if test_image is None:
        print(f"Erro: Não foi possível carregar a imagem de {TEST_IMAGE_PATH}")
    else:
        # Estima a profundidade
        depth_map = estimator.estimate(test_image)
        
        if depth_map is not None:
            print("Mapa de profundidade gerado com sucesso.")
            # Exibe o mapa de profundidade
            plt.figure(figsize=(10, 8))
            plt.imshow(depth_map, cmap='gray')
            plt.title("Mapa de Profundidade (MiDaS)")
            plt.axis('off')
            plt.show()
        else:
            print("Falha ao gerar o mapa de profundidade.")

## 3. Teste do Reconhecimento Facial (DeepFace)

In [ ]:
# Carrega o serviço de reconhecimento facial
try:
    recognizer = FaceRecognizer(settings_dict)
    print("FaceRecognizer carregado com sucesso.")
except Exception as e:
    print(f"Falha ao carregar FaceRecognizer: {e}")
    recognizer = None

if recognizer:
    # Carrega a imagem de teste (deve conter um rosto)
    face_image = cv2.imread(str(TEST_IMAGE_PATH))
    if face_image is None:
        print(f"Erro: Não foi possível carregar a imagem de {TEST_IMAGE_PATH}")
    else:
        print("Tentando identificar rostos na imagem de teste...")
        # Tenta identificar rostos na imagem contra o banco de dados existente
        identified_dfs = recognizer.identify_faces(face_image)
        
        if not identified_dfs:
            print("Nenhum rosto conhecido foi identificado na imagem.")
        else:
            print("Rostos conhecidos identificados:")
            for df in identified_dfs:
                print(df)
        
        show_image(face_image, title="Imagem de Teste para Reconhecimento Facial")